In [ ]:
import sqlite3
import os
import pandas as pd
import numpy as np
from data_functions import *

# Compile RB

In [ ]:
#==========
# Load team-based statistics
#==========

'''
Pull in the oline, quarterback, and overall team offense statistics, and join them 
to the player data. This will provide team-based context for the players, as well as
allow for grouped statistics generation.
'''

# load prepared data
conn = sqlite3.connect('/Users/Mark/Documents/Github/Fantasy_Football/Data/Season_Stats.sqlite3')
query = ''' 
    SELECT * 
    FROM RB_Stats A 
    INNER JOIN OLine_Stats B ON A.team = B.team AND A.year = B.year
    INNER JOIN Team_Efficiency C ON A.team = C.team AND A.year = C.year
    INNER JOIN Team_Offensive_Stats D ON A.team = D.team AND A.year = D.year
    INNER JOIN QB_PosPred E ON A.team = E.team AND A.year = E.year'''

df = pd.read_sql_query(query, con=conn)

# remove duplicated columns
df = df.loc[:, ~df.columns.duplicated()]

# ensure everything is numeric
for col in df.columns:
    try:
        df[col] = df[col].astype('float')
    except:
        pass

In [ ]:
#==========
# Creating team based grouped statistics
#==========

'''
Create grouped statistics based on the team and teammates. For example,
create total touches by team feature, as well as how the average, min, and max
teammate adps compare to the current player.
'''

# groupby team and year to get total rb touches for each team
team_touches = df.groupby(['team', 'year'], group_keys=False)['rush_att'].agg(np.sum).reset_index().rename(columns={'rush_att': 'rb_att_on_team'})
df = pd.merge(df, team_touches, how='left', left_on=['team', 'year'], right_on=['team', 'year'])
df['available_rush_att'] = 1-(df['rb_att_on_team'] / df['tm_rush_att'])
df['available_rush_att_2'] = 1-((df['rb_att_on_team'] - df['att']) / df['tm_rush_att'])

team_tgts = df.groupby(['team', 'year'], group_keys=False)['tgt'].agg(np.sum).reset_index().rename(columns={'tgt': 'tgt_on_team'})
df = pd.merge(df, team_tgts, how='left', left_on=['team', 'year'], right_on=['team', 'year'])
df['available_tgt'] = 1-(df['tgt_on_team'] / df['tm_pass_att'])
df['available_tgt_2'] = 1-((df['tgt_on_team'] - df['tgt']) / df['tm_pass_att'])

# create market share statistics
df['ms_rush_att'] = df['att'] / df['tm_rush_att']
df['ms_rush_yd'] = df['rush_yds'] / df['tm_rush_yds']
df['ms_rush_td'] = df['rush_td'] / df['tm_rush_td']
df['ms_rec_yd'] = df['rec_yds'] / df['tm_pass_yds']
df['ms_tgts'] = df['tgt'] / df['tm_pass_att']

df['ms_rush_yd_per_att'] = df['ms_rush_yd'] / df['ms_rush_att']

df['avail_x_newteam'] = df['available_rush_att'] * df['new_team']

# create teammate ADP metrics to see if top ranked player
min_teammate = df.groupby(['team', 'year'], group_keys=False)['avg_pick'].agg(np.min).reset_index().rename(columns={'avg_pick': 'min_teammate'})
max_teammate = df.groupby(['team', 'year'], group_keys=False)['avg_pick'].agg(np.max).reset_index().rename(columns={'avg_pick': 'max_teammate'})
avg_teammate = df.groupby(['team', 'year'], group_keys=False)['avg_pick'].agg(np.mean).reset_index().rename(columns={'avg_pick': 'avg_teammate'})

names = ['min_teammate', 'max_teammate', 'avg_teammate']
for i, file in enumerate([min_teammate, max_teammate, avg_teammate]):
    
    df = pd.merge(df, file, how='inner', left_on=['team', 'year'], right_on=['team', 'year'])
    df['teammate_diff'] = df[names[i]] - df['min_teammate'] 

In [ ]:
append_to_db(df, db_name='Model_Inputs.sqlite3', table_name='RB_2018', if_exist='replace')

# Compile WR

In [ ]:
#==========
# Load team-based statistics
#==========

'''
Pull in the oline, quarterback, and overall team offense statistics, and join them 
to the player data. This will provide team-based context for the players, as well as
allow for grouped statistics generation.
'''

# load prepared data
conn = sqlite3.connect('/Users/Mark/Documents/Github/Fantasy_Football/Data/Season_Stats.sqlite3')
query = ''' 
    SELECT * 
    FROM WR_Stats A 
    INNER JOIN OLine_Stats B ON A.team = B.team AND A.year = B.year
    INNER JOIN Team_Efficiency C ON A.team = C.team AND A.year = C.year
    INNER JOIN Team_Offensive_Stats D ON A.team = D.team AND A.year = D.year
    INNER JOIN QB_PosPred E ON A.team = E.team AND A.year = E.year'''

df = pd.read_sql_query(query, con=conn)

# remove duplicated columns
df = df.loc[:, ~df.columns.duplicated()]

# ensure everything is numeric
for col in df.columns:
    try:
        df[col] = df[col].astype('float')
    except:
        pass

In [ ]:
#==========
# Creating team based grouped statistics
#==========

'''
Create grouped statistics based on the team and teammates. For example,
create total touches by team feature, as well as how the average, min, and max
teammate adps compare to the current player.
'''

# groupby team and year to get total wr yard and targets for each team
team_tgts = df.groupby(['team', 'year'], group_keys=False)['tgt'].agg(np.sum).reset_index().rename(columns={'tgt': 'tgt_on_team'})
df = pd.merge(df, team_tgts, how='inner', left_on=['team', 'year'], right_on=['team', 'year'])
df['available_tgt'] = 1-(df['tgt_on_team'] / df['tm_pass_att'])
df['available_tgt_2'] = 1-((df['tgt_on_team'] - df['tgt']) / df['tm_pass_att'])

team_yds = df.groupby(['team', 'year'], group_keys=False)['rec_yds'].agg(np.sum).reset_index().rename(columns={'rec_yds': 'yds_on_team'})
df = pd.merge(df, team_yds, how='inner', left_on=['team', 'year'], right_on=['team', 'year'])
df['available_yds'] = 1-(df['yds_on_team'] / df['tm_pass_yds'])
df['available_yds_2'] = 1-((df['yds_on_team']-df['rec_yds']) / df['tm_pass_yds'])

# create marketshare statistics
df['ms_rec_yd'] = df['rec_yds'] / df['tm_pass_yds']
df['ms_tgts'] = df['tgt'] / df['tm_pass_att']
df['ms_yds_per_tgts'] = df['ms_rec_yd'] / df['ms_tgts']

df['avail_tgt_x_newteam'] = df['available_tgt'] * df['new_team']
df['avail_yds_x_newteam'] = df['available_yds'] * df['new_team']

# create teammate ADP metrics to see if top ranked player
min_teammate = df.groupby(['team', 'year'], group_keys=False)['avg_pick'].agg(np.min).reset_index().rename(columns={'avg_pick': 'min_teammate'})
max_teammate = df.groupby(['team', 'year'], group_keys=False)['avg_pick'].agg(np.max).reset_index().rename(columns={'avg_pick': 'max_teammate'})
avg_teammate = df.groupby(['team', 'year'], group_keys=False)['avg_pick'].agg(np.mean).reset_index().rename(columns={'avg_pick': 'avg_teammate'})

names = ['min_teammate', 'max_teammate', 'avg_teammate']
for i, file in enumerate([min_teammate, max_teammate, avg_teammate]):
    
    df = pd.merge(df, file, how='inner', left_on=['team', 'year'], right_on=['team', 'year'])
    df['teammate_diff'] = df[names[i]] - df['min_teammate'] 

In [ ]:
append_to_db(df, db_name='Model_Inputs.sqlite3', table_name='WR_2018', if_exist='replace')